In [1]:
# This chunk of code make it possible to use src functions
import sys
import os
module_path = os.path.abspath(os.path.join('../src'))

if module_path not in sys.path:
    sys.path.append(module_path)
import warnings
warnings.filterwarnings('ignore')

from dsproject import dsproject
from preprocess import *

dsp = dsproject()

In [2]:
import pandas as pd

db_soc = dsp.read_table('DB_SOC', 'preprocess', index_col=0)
db_soc = db_soc.loc[db_soc['Group'].isin([1,3,5])]
db_soc = db_soc.loc[db_soc['OSAMT'] > 100]

In [3]:
db_soc['FileDate'] = parse_dates(db_soc['FileDate'], format='%Y%m%d')

In [4]:
#Generate unique filedate table
unique_filedate = db_soc[['BILLCycle', 'FileDate']].drop_duplicates()

unique_filedate['Next_FileDate'] = unique_filedate.groupby('BILLCycle')[['BILLCycle','FileDate']].shift(-1)
unique_filedate['Previous_FileDate'] = unique_filedate.groupby('BILLCycle')[['BILLCycle','FileDate']].shift(1)
db_soc = db_soc.merge(unique_filedate, on=['BILLCycle', 'FileDate'])

unique_filedate = db_soc[['HASH_AR_ID', 'FileDate', 'Previous_FileDate', 'Next_FileDate']].drop_duplicates()
dsp.write_table(unique_filedate, 'unique_filedate', 'feature')

In [5]:
# add HASH_IP_ID
import numpy as np
df_id = dsp.read_data('EDW_LPM_X_AR_X_CIS', usecols=['HASH_IP_ID','HASH_AR_ID'], dtype={'HASH_IP_ID':str,'HASH_AR_ID':np.int64})
df = db_soc.merge(df_id, on=['HASH_AR_ID'], how ='inner')
print("Merge coverage")
print(len(df.index)/len(db_soc.index))

Merge coverage
1.0000621329980006


In [6]:
df = df[['HASH_AR_ID', 'HASH_IP_ID', 'HASH_LPM_CST_ID', 'FileDate', 'Previous_FileDate', 'Next_FileDate', 'Group', 'OSAMT']]

In [7]:
dsp.write_table(df, 'group_target', 'feature', ds_type=['id', 'id', 'id', 'date', 'date', 'date', 'category', 'numeric'])

In [8]:
len(df.index)

1078399